In [8]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [9]:
#Load and view dataset
df = pd.read_excel('batching_impact.xlsx', sheet_name='Combined Sheet')
df.head()

,Area,Date,Day,hour,OCARA,COBRA,COBRM,G2N,ETA,FM,Expiry %,total,hourly_gross_captains,20_sec_gross_captains,rides_per_hour,rides_per_20_sec,FM_decrease %,FM_decrease Absolute,dropped_rides_in,dropped_rides_out,replenishment
0,bangalore_area1,19/09/2022,Monday,0,0.3389,0.3863,0.0024,0.2441,4.29,1.37,0.0284,422,2971,16.51,422,2.34,-0.0273,0.037401,93.0,103.0,-10.0
1,bangalore_area1,19/09/2022,Monday,1,0.3418,0.3367,0.0000,0.2857,4.19,1.33,0.0306,196,1184,6.58,195,1.08,0.0032,0.004256,47.0,56.0,-9.0
2,bangalore_area1,19/09/2022,Monday,2,0.2885,0.3558,0.0000,0.3269,4.55,1.54,0.0192,104,690,3.83,104,0.58,0.0701,0.107954,31.0,34.0,-3.0
3,bangalore_area1,19/09/2022,Monday,3,0.3210,0.3704,0.0000,0.2469,4.58,1.55,0.0494,81,517,2.87,81,0.45,0.0221,0.034255,18.0,20.0,-2.0
4,bangalore_area1,19/09/2022,Monday,4,0.2034,0.5254,0.0000,0.1610,5.81,1.82,0.1102,118,727,4.04,118,0.66,-0.0004,0.000728,16.0,19.0,-3.0


In [10]:
#Filter on times having negative replenishment
df_replenishment_neg = df[df['replenishment'] < 0]
df_replenishment_neg = df_replenishment_neg[df_replenishment_neg['hour'].isin([8, 9, 10, 18, 19, 20])]
df_replenishment_neg.head()

,Area,Date,Day,hour,OCARA,COBRA,COBRM,G2N,ETA,FM,Expiry %,total,hourly_gross_captains,20_sec_gross_captains,rides_per_hour,rides_per_20_sec,FM_decrease %,FM_decrease Absolute,dropped_rides_in,dropped_rides_out,replenishment
9,bangalore_area1,19/09/2022,Monday,9,0.0652,0.3236,0.1830,0.1211,4.56,1.27,0.3065,3344,13072,72.62,3331,18.51,-0.1085,0.137795,396.0,405.0,-9.0
10,bangalore_area1,19/09/2022,Monday,10,0.0898,0.3663,0.1126,0.1883,4.63,1.39,0.2427,2629,13262,73.68,2623,14.57,-0.0889,0.123571,446.0,495.0,-49.0
32,bangalore_area1,20/09/2022,Tuesday,8,0.0995,0.3393,0.0797,0.2639,4.76,1.37,0.2163,1618,9612,53.40,1612,8.96,-0.1098,0.150426,405.0,427.0,-22.0
33,bangalore_area1,20/09/2022,Tuesday,9,0.0645,0.3579,0.1369,0.1537,4.41,1.33,0.2852,3163,13871,77.06,3159,17.55,-0.1232,0.163856,465.0,486.0,-21.0
34,bangalore_area1,20/09/2022,Tuesday,10,0.1147,0.3998,0.0417,0.2945,4.35,1.30,0.1489,2136,14522,80.68,2128,11.82,-0.1141,0.148330,550.0,629.0,-79.0


In [11]:
#Groupby on area to get Median OCARA, COBRA, G2N and Expiry
metrics = df_replenishment_neg.groupby(['Area'])['OCARA', 'COBRA', 'G2N', 'Expiry %', 'replenishment'].median().reset_index()
metrics

,Area,OCARA,COBRA,G2N,Expiry %,replenishment
0,bangalore_area1,0.1154,0.34045,0.34650,0.12750,-43.5
1,bangalore_area2,0.1481,0.37720,0.34165,0.08060,-13.0
2,delhi_area1,0.2006,0.14325,0.57840,0.05835,-69.0
3,delhi_area2,0.1922,0.11250,0.68980,0.00350,-95.0
4,hyderabad_area1,0.2314,0.08125,0.64765,0.01290,-50.5
5,hyderabad_area2,0.2100,0.07520,0.68440,0.00000,-25.5
